In [5]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX

In [6]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [20]:
includet("utils.jl")
includet("turing_nw_bvar.jl")

In [10]:
df = DataFrame(XLSX.readtable("../../data/usa_world_data.xlsx", "Arkusz1", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

146×15 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-03-01 to 2024-04-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1988-03-01 │   0.0857 │   677.401 │       262.6 │  9.04333 │  18.59 │ 14.433 ⋯
│ 1988-06-01 │   0.0882 │   681.206 │       269.1 │  9.30667 │  21.67 │ 14.608 ⋯
│ 1988-09-01 │   0.0887 │   694.387 │       277.4 │     9.55 │  22.73 │ 14.811 ⋯
│ 1988-12-01 │   0.0914 │   697.568 │       285.4 │  9.81333 │  23.75 │ 15.088 ⋯
│ 1989-03-01 │    0.093 │   706.377 │       302.3 │  10.0867 │  24.96 │ 15.686 ⋯
│ 1989-06-01 │    0.081 │    780.73 │       331.9 │  10.4233 │  25.22 │ 17.013 ⋯
│ 1989-09-01 │   0.0831 │   785.343 │       347.4 │  10.7967 │  23.69 │ 17.640 ⋯
│ 1989-12-01 │   0.0793 │   821.683 │      339.97 │    11.14 │  22.87 │ 17.048 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [40]:
freq = 1
#:Price_10Y
returns = percentchange(data_source[:Total_Stock, :Price_10Y, :World_ex_USA, :EM_Net], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
#excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
data = merge(excess_returns, real_t_bill, term, cpi, dy)


start = Date(1989, 01,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))

display("Correlations")
display(cor(values(to(from(real_returns, start),end_d))))

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.0714,0.1957,0.0253,0.0662,-0.2537,-0.8536,-0.0039,0.0712,1.8048,0.354,0.0357
Price_10Y,0.0418,0.1423,0.0133,0.0423,-0.0725,0.3136,-0.0175,0.0162,0.0266,0.3188,0.0078
World_ex_USA,0.0838,0.3042,0.0116,0.0656,-0.4449,-0.9618,-0.0376,0.124,5.7703,0.1386,0.0189
EM_Net,0.1265,0.446,0.02,0.0954,-0.5904,-0.8653,-0.0484,0.0758,4.3012,0.1584,0.0322


nothing

"real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.0714,0.1938,0.0186,0.061,-0.2381,-0.728,-0.011,0.0581,1.3742,0.2603,0.0283
Price_10Y_CPI,0.0447,0.1677,0.0066,0.0367,-0.096,0.3686,-0.0257,0.0313,0.5634,0.1488,0.003
World_ex_USA_CPI,0.082,0.2943,0.0049,0.0565,-0.4195,-0.7637,-0.0414,0.1193,4.7076,0.0602,0.0116
EM_Net_CPI,0.1246,0.4361,0.0134,0.0881,-0.565,-0.765,-0.0561,0.0713,3.8365,0.1072,0.0166


nothing

"Correlations"

4×4 Matrix{Float64}:
  1.0        -0.0970977   0.45003    0.316977
 -0.0970977   1.0        -0.357511  -0.405884
  0.45003    -0.357511    1.0        0.716905
  0.316977   -0.405884    0.716905   1.0

In [41]:
T, K = size(real_returns)

model = mvnormal_lkjcholesky_model(values(real_returns), K, T)

# Sample from the posterior
chain = sample(model, NUTS(), MCMCThreads(), 1000, 4; discard_initial=1000)

# Summarize results
display(summarystats(chain))
display(quantile(chain))


Sampling (4 threads)   0%|█                             |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.003125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.0001953125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
Sampling (4 threads)  25%|████████                      |  ETA: 0:00:28
Sampling (4 threads)  50%|████████████████              |  ETA: 0:00:09
Sampling (4 threads)  75%|███████████████████████       |  ETA: 0:00:03
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:00:10
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:00:10


Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        μ[1]    0.0189    0.0059    0.0001   5432.4936   3281.0427    1.0013   ⋯
        μ[2]    0.0059    0.0038    0.0001   5822.1992   3647.3465    1.0021   ⋯
        μ[3]    0.0061    0.0070    0.0001   4163.5896   3600.2420    1.0023   ⋯
        μ[4]    0.0144    0.0104    0.0002   4568.6166   3392.5135    1.0012   ⋯
        σ[1]    0.0713    0.0042    0.0001   5379.0344   3332.7003    1.0033   ⋯
        σ[2]    0.0449    0.0027    0.0000   5277.7992   3456.6862    1.0023   ⋯
        σ[3]    0.0826    0.0047    0.0001   2940.1336   3026.7315    1.0010   ⋯
        σ[4]    0.1242    0.0073    0.0001   3468.7748   2954.3670    1.0010   ⋯
   L.L[1, 1]    1.0000    0.0000       NaN         NaN         NaN       NaN   ⋯
   L.L[2, 1]   -0.0888    0.0830    0.0013   3944.1704   3273.6870    1.0007   ⋯
   L.L[3

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

        μ[1]    0.0075    0.0149    0.0189    0.0229    0.0300
        μ[2]   -0.0016    0.0032    0.0060    0.0086    0.0134
        μ[3]   -0.0079    0.0015    0.0061    0.0109    0.0197
        μ[4]   -0.0055    0.0074    0.0143    0.0215    0.0349
        σ[1]    0.0636    0.0683    0.0710    0.0740    0.0802
        σ[2]    0.0401    0.0430    0.0447    0.0465    0.0505
        σ[3]    0.0739    0.0794    0.0826    0.0856    0.0924
        σ[4]    0.1109    0.1192    0.1238    0.1289    0.1393
   L.L[1, 1]    1.0000    1.0000    1.0000    1.0000    1.0000
   L.L[2, 1]   -0.2499   -0.1463   -0.0887   -0.0342    0.0746
   L.L[3, 1]    0.2911    0.3873    0.4327    0.4749    0.5541
   L.L[4, 1]    0.1477    0.2489    0.3004    0.3506    0.4366
   L.L[2, 2]    0.9683    0.9892    0.9958    0.9989    1.0000
   L.L[3, 2]   -0.4342   -0.3530   -0.3075

In [52]:
μ = mean(group(chain, :μ)).nt.mean
σ = mean(group(chain, :σ)).nt.mean
L = mean(group(chain, "L.L")).nt.mean

Σ = est_covariance(σ, L)

n_samples = 10000
n_steps = 100
scenarios = zeros(K, n_steps, n_samples)

for t in 1:n_steps
    scenarios[:,t,:]= rand(MvNormal(μ, Σ), 10000)
end

for k in 1:K
    display(round.(quantile(scenarios[k,1,:], [.025, .25, .5, .75, .975]), digits=2))
end


5-element Vector{Float64}:
 -0.12
 -0.03
  0.02
  0.07
  0.16

5-element Vector{Float64}:
 -0.08
 -0.02
  0.01
  0.04
  0.09

5-element Vector{Float64}:
 -0.15
 -0.05
  0.01
  0.06
  0.16

5-element Vector{Float64}:
 -0.22
 -0.07
  0.01
  0.1
  0.25

#### TODO
- est cumr returns, 1, 5, 30
- display table mean, std, skewness, kurtosis, for every series, every time step
- display correlation matrix for every time step
